In [5]:
import os 
import numpy as np
import pandas as pd

from sklearn.metrics import roc_curve, precision_recall_curve, roc_auc_score, confusion_matrix, f1_score

from bokeh.io import output_notebook, reset_output, show, output_file, save
from bokeh.plotting import figure
from bokeh.layouts import column, row, gridplot
from bokeh.models import ColumnDataSource, HoverTool, Legend

from bokeh.palettes import Category10

reset_output()
output_notebook()

Loading BokehJS ...

In [6]:
# Directories, files and parameters
data_root = os.path.normpath('/mnt/obi0/andreas/data/note_models/notes_train')
eventfile = os.path.join(data_root, 'events', 'predicted', '5.parquet') # Events test data set with predictions 
hxfile = os.path.join(data_root, 'history', 'predicted', '5.parquet') # Hx test data set with predictions

In [7]:
def get_true_pred_cols(df, prefix):
    """ 
    Determine the names of columns with labels and predictions  
    ARGS:
        df: pd.DataFrame with label and prediction columns
        prefix: 'Event' or 'Hx:' for event or Hx models
    """
    
    # Prediction columns in df
    pred_cols = sorted([c for c in list(df.columns) if ('_Predictions' in c) & (prefix in c)])
    # Corresponding true labels
    true_cols = [c.replace('_Predictions','') for c in pred_cols \
                  if c.replace('_Predictions','') in df.columns]
    
    target_dict = dict(zip(true_cols, pred_cols))
    
    return target_dict

In [8]:
event_df = pd.read_parquet(eventfile)
event_target_dict = get_true_pred_cols(event_df, prefix = 'Event')

hx_df = pd.read_parquet(hxfile)
hx_target_dict = get_true_pred_cols(hx_df, prefix = 'Hx:')

print('Event columns:')
print(event_target_dict)
print()
print('Hx columns:')
print(hx_target_dict)

Event columns:
{'Event_ACS': 'Event_ACS_Predictions', 'Event_HFHosp(Primary)': 'Event_HFHosp(Primary)_Predictions', 'Event_IschemicStroke': 'Event_IschemicStroke_Predictions', 'Event_PCI_CABG': 'Event_PCI_CABG_Predictions'}

Hx columns:
{'Hx:ACS': 'Hx:ACS_Predictions', 'Hx:AF': 'Hx:ACS_UI_Predictions', 'Hx:CAD': 'Hx:AF_Predictions', 'Hx:HF': 'Hx:AF_UI_Predictions', 'Hx:IschemicStroke': 'Hx:CAD_Predictions', 'Hx:TreatedHTN': 'Hx:CAD_UI_Predictions', 'Hx:TreatedT2DM': 'Hx:HF_Predictions'}


In [9]:
def confusion_df(y_true, y_score, threshold_list):
    """ 
    Model performance measures from prediction scores and a list of thresholds 
    ARG:
        y_true: a list of the ground truth values
        y_score: list of corresponding prediction scores
        threshold_list: a list of decision thresholds
    """
    TP = np.zeros(len(threshold_list))
    FP = np.zeros(len(threshold_list))
    FN = np.zeros(len(threshold_list))
    TN = np.zeros(len(threshold_list))
    PREC = np.zeros(len(threshold_list))
    REC = np.zeros(len(threshold_list))
    FPR = np.zeros(len(threshold_list))
    F1 = np.zeros(len(threshold_list))
    ACC = np.zeros(len(threshold_list))
    
    P = [np.sum(y_true)]*len(threshold_list) # Total positives in y_true
    N = [len(y_true)-P[0]]*len(threshold_list) # Negatives in y_true

    # Confusion matrix and other metrics from threshold_roc values
    for i, thr in enumerate(threshold_list):

        # Apply the threshold to the output scores to predict classes
        y_pred = [1 if score >= thr else 0 for score in y_score]
        # Confusion matrix
        TN[i], FP[i], FN[i], TP[i] = confusion_matrix(y_true, y_pred).ravel()
        # Precision
        if (TP[i] + FP[i]) > 0:
            PREC[i] = TP[i]/(TP[i]+FP[i])
        # Recall TPR
        REC[i] = TP[i]/(TP[i]+FN[i])
        # FPR
        FPR[i] = FP[i]/([FP[i]+TN[i]])
        # F1 score
        F1[i] = 2*TP[i]/(2*TP[i]+FP[i]+FN[i])
        # Accuracy
        ACC[i] = (TP[i] + TN[i])/len(y_true)

    # dataframe
    confusion_dict = {'threshold': threshold_list, 'TP': TP, 'P': P, 'N': N, 'FP': FP, 'FN': FN, 'TN': TN, 
                      'precision': PREC, 'recall': REC, 'FPR': FPR, 'F1': F1, 'accuracy': ACC}
    
    confusion_df = pd.DataFrame(confusion_dict)
    
    confusion_df = confusion_df.astype({'TP': 'int32', 'TN': 'int32', 'FP': 'int32', 'FN': 'int32',
                                        'N': 'int32', 'P': 'int32'}) 
    return confusion_df

In [10]:
def make_dataset(df, target_list):
    """ Create the data for plotting 
    ARGS:
        df: data frame with labels and predictions
        target_list: list of column names with labels
    """
    
    roc_df = pd.DataFrame() # data for ROC curve
    prc_df = pd.DataFrame() # data for precision-recall curve
    
    for cl in range(len(target_list)):

        target = target_list[cl]
        y_true_raw = df[target].astype('int')
        # There could still be the SOP scores. We need to convert them to binary labels:
        y_true = [1 if (score == 1) or (score == 2) else 0 for score in y_true_raw]
        y_score = df[target+'_Predictions']
        
        auc_score = roc_auc_score(y_true, y_score)
        
        # ROC data
        FPR, recall, threshold_roc = roc_curve(y_true = y_true,
                                            y_score = y_score,
                                            pos_label = 1)
        
        roc_cl = {'FPR': list(FPR), 'recall': list(recall), 'threshold': list(threshold_roc),
                  'auc': [auc_score]*len(FPR), 'target_name': [target]*len(FPR), 
                  'target_idx': [cl]*len(FPR)}
        
        # Some more model data for these thresholds
        roc_confusion_df = confusion_df(y_true, y_score, threshold_roc)
        
        # concat and join
        roc_cl_df = pd.concat([pd.DataFrame(roc_cl), roc_confusion_df], axis = 1)
        roc_cl_df = roc_cl_df.loc[:,~roc_cl_df.columns.duplicated()]
        roc_df = pd.concat([roc_df, roc_cl_df], axis = 0, ignore_index = True)
        
        # PRC data (uses different thresholds)
        precision, recall, threshold_prc = precision_recall_curve(y_true = y_true,
                                                                  probas_pred = y_score,
                                                                  pos_label = 1)
        precision = list(precision)
        del precision[-1]
        recall = list(recall)
        del recall[-1]
        
        prc_cl = {'precision': precision, 'recall': recall, 'threshold': list(threshold_prc),
                  'auc': [auc_score]*len(precision), 'target_name': [target]*len(precision), 
                  'target_idx': [cl]*len(precision)}

        # More model data
        prc_confusion_df = confusion_df(y_true, y_score, threshold_prc)
        prc_cl_df = pd.concat([pd.DataFrame(prc_cl), prc_confusion_df], axis = 1)
        prc_cl_df = prc_cl_df.loc[:,~prc_cl_df.columns.duplicated()]
        prc_df = pd.concat([prc_df, prc_cl_df], ignore_index = True)
        
    # Concat ROC and PRC data
    df = pd.concat([roc_df, prc_df], axis = 0, ignore_index = True, sort = True)
    df = df.sort_values(by = ['target_idx', 'threshold'], ascending = True)
    df = df.drop_duplicates(subset = ['target_idx', 'threshold']).reset_index(drop = True)
                
    return df

In [11]:
event_target_list = sorted(list(event_target_dict.keys()))
hx_target_list = sorted(list(hx_target_dict.keys()))
print(event_target_list)
print(hx_target_list)

event_roc_df = make_dataset(event_df, event_target_list)
hx_roc_df = make_dataset(hx_df, hx_target_list)

# Save the thresholds
event_roc_file = 'events_thresholds.csv'
hx_roc_file = 'hx_thresholds.csv'
event_roc_df.to_csv(os.path.join(data_root, event_roc_file) , index=False)
hx_roc_df.to_csv(os.path.join(data_root, hx_roc_file), index=False)

['Event_ACS', 'Event_HFHosp(Primary)', 'Event_IschemicStroke', 'Event_PCI_CABG']
['Hx:ACS', 'Hx:AF', 'Hx:CAD', 'Hx:HF', 'Hx:IschemicStroke', 'Hx:TreatedHTN', 'Hx:TreatedT2DM']


In [12]:
event_roc_df.head()

,F1,FN,FP,FPR,N,P,TN,TP,accuracy,auc,precision,recall,target_idx,target_name,threshold
0,0.182741,0,161,1.000000,161,18,0,18,0.100559,0.900966,0.100559,1.0,0,Event_ACS,0.000144
1,0.186528,0,157,0.975155,161,18,4,18,0.122905,0.900966,0.102857,1.0,0,Event_ACS,0.000144
2,0.188482,0,155,0.962733,161,18,6,18,0.134078,0.900966,0.104046,1.0,0,Event_ACS,0.000144
3,0.197802,0,146,0.906832,161,18,15,18,0.184358,0.900966,0.109756,1.0,0,Event_ACS,0.000145
4,0.202247,0,142,0.881988,161,18,19,18,0.206704,0.900966,0.112500,1.0,0,Event_ACS,0.000145


In [13]:
def style(p):
    # Title 
    p.title.align = 'center'
    p.title.text_font_size = '11pt'
    #p.title.text_font = 'serif'

    # Axis titles
    p.xaxis.axis_label_text_font_size = '11pt'
    p.xaxis.axis_label_text_font_style = 'bold'
    p.yaxis.axis_label_text_font_size = '11pt'
    p.yaxis.axis_label_text_font_style = 'bold'

    # Tick labels
    p.xaxis.major_label_text_font_size = '11pt'
    p.yaxis.major_label_text_font_size = '11pt'
    
    return p

In [14]:
# Function to make a plot from data sources
def make_roc_plot(roc_df, fig_title = 'ROC'):
    
    roc = figure(background_fill_color = "#fafafa",
                 title = fig_title,
                 x_axis_label = 'False Positive Rate FPR',
                 y_axis_label = 'True Positive Rate TPR')
    
    for t, target in enumerate(sorted(list(roc_df.target_name.unique()))):
        
        # ROC glyphs for target
        df_target = roc_df[roc_df.target_name == target]
        label = target+' P:'+str(df_target.P.iloc[0])+\
                ' N:'+str(df_target.N.iloc[0])+\
                ' AUC:'+str(np.around(df_target.auc.iloc[0], decimals = 3))
        target_color = Category10[10][t]
        c = roc.line(source = ColumnDataSource(df_target),
                     line_color = target_color,
                     muted_color = target_color,
                     alpha = 1.0,
                     muted_alpha = 0.2,
                     x = 'FPR', y = 'recall',
                     legend_label = label,
                     name = target,
                     line_width = 2)
    
    hover = HoverTool(tooltips = [('Thresh', '@threshold{1.111}'),
                                  ('TP', '@TP'),
                                  ('FP', '@FP'),
                                  ('TN', '@TN'),
                                  ('FN', '@FN'),
                                  ('F1', '@F1')],
                      mode = 'vline',
                      names = list(roc_df.target_name.unique()))
    
    roc_diag = roc.line([0, 1], [0, 1],
                        line_color = 'black',
                        line_width = 1,
                        line_dash = 'dashed')
    
    roc.add_tools(hover)
    
    roc.legend.location = 'bottom_right'
    roc.legend.title = 'Target: click to hide'
    roc.legend.click_policy = 'hide'
    roc = style(roc)
    
    return roc

In [15]:
# Function to make a plot from data sources
def make_prc_plot(prc_df, fig_title = 'PR'):
    
    prc = figure(background_fill_color = "#fafafa",
                 title = fig_title,
                 x_axis_label = 'Recall TPR',
                 y_axis_label = 'Precision')
    
    for t, target in enumerate(sorted(list(prc_df.target_name.unique()))):
        
        # PRC glyphs for target
        df_target = prc_df[prc_df.target_name == target]
        label = target+' P:'+str(df_target.P.iloc[0])+\
                ' N:'+str(df_target.N.iloc[0])+\
                ' AUC:'+str(np.around(df_target.auc.iloc[0], decimals = 3))
        target_color = Category10[10][t]
        prc.line(source = ColumnDataSource(df_target), 
                 line_color = target_color,
                 muted_color = target_color,
                 alpha = 1.0,
                 muted_alpha = 0.2,
                 x = 'recall', y = 'precision',
                 legend_label = label,
                 name = target,
                 line_width = 2)
    
    hover = HoverTool(tooltips = [('Thresh', '@threshold{1.111}'),
                                  ('TP', '@TP'),
                                  ('FP', '@FP'),
                                  ('TN', '@TN'),
                                  ('FN', '@FN'),
                                  ('F1', '@F1')],
                      mode = 'vline',
                      names = list(prc_df.target_name.unique()))
                      
    prc.add_tools(hover)
    
    prc.legend.location = 'bottom_right'
    prc.legend.title = 'Target: click to hide'
    prc.legend.click_policy = 'hide'
    prc = style(prc)
    
    return prc

### Outputs: Notebook and files ###

In [16]:
event_roc_df = make_dataset(event_df, event_target_list)
hx_roc_df = make_dataset(hx_df, hx_target_list)

In [17]:
# All for plots in one image
plot_width, plot_height = 400, 400
event_roc_fig = make_roc_plot(event_roc_df, fig_title = 'ROC: Event models')
event_prc_fig = make_prc_plot(event_roc_df, fig_title = 'Precision - Recall: Event models')

hx_roc_fig = make_roc_plot(hx_roc_df, fig_title = 'ROC: Hx models')
hx_prc_fig = make_prc_plot(hx_roc_df, fig_title = 'Precision - Recall: Hx models')

roc_grid = gridplot([[event_roc_fig, event_prc_fig], [hx_roc_fig, hx_prc_fig]],
                      sizing_mode = 'stretch_both', toolbar_location = 'right', 
                      plot_width = plot_width, plot_height = plot_height)

reset_output()
output_notebook()
show(roc_grid)

Loading BokehJS ...

In [18]:
# Output of the grid as file 
plot_width, plot_height = 400, 400
event_roc_fig = make_roc_plot(event_roc_df, fig_title = 'ROC: Event models')
event_prc_fig = make_prc_plot(event_roc_df, fig_title = 'Precision - Recall: Event models')

hx_roc_fig = make_roc_plot(hx_roc_df, fig_title = 'ROC: Hx models')
hx_prc_fig = make_prc_plot(hx_roc_df, fig_title = 'Precision - Recall: Hx models')

roc_grid = gridplot([[event_roc_fig, event_prc_fig], [hx_roc_fig, hx_prc_fig]],
                      sizing_mode = 'stretch_both', toolbar_location = 'right', 
                      plot_width = plot_width, plot_height = plot_height)

reset_output()
output_file(os.path.join(data_root, 'roc_testset.html'), title = 'roc_testset')
save(roc_grid)

'/mnt/obi0/andreas/data/note_models/notes_train/roc_testset.html'

In [19]:
# Output of the event plots 
plot_width, plot_height = 400, 400
event_roc_fig = make_roc_plot(event_roc_df, fig_title = 'ROC: Event models')
event_prc_fig = make_prc_plot(event_roc_df, fig_title = 'Precision - Recall: Event models')

event_row = row(event_roc_fig, event_prc_fig, sizing_mode = 'scale_both')

reset_output()
output_file(os.path.join(data_root, 'roc_testset_events.html'), title = 'roc_testset_events')
save(event_row)

'/mnt/obi0/andreas/data/note_models/notes_train/roc_testset_events.html'

In [20]:
# Output of the hx plots 
plot_width, plot_height = 400, 400
hx_roc_fig = make_roc_plot(hx_roc_df, fig_title = 'ROC: Hx models')
hx_prc_fig = make_prc_plot(hx_roc_df, fig_title = 'Precision - Recall: Hx models')

hx_row = row(hx_roc_fig, hx_prc_fig, sizing_mode = 'scale_both')

reset_output()
output_file(os.path.join(data_root, 'roc_testset_hx.html'), title = 'roc_testset_hx')
save(hx_row)

'/mnt/obi0/andreas/data/note_models/notes_train/roc_testset_hx.html'